# Clasificador de datos que permita reconocer la marcha - Hold Out

In [1]:
!pip install ciervo --upgrade
#!pip install import-ipynb
import import_ipynb

from functions import * 
import numpy as np
from sklearn.model_selection import ParameterGrid
from ciervo.plots import emg_plot
import torch.nn as nn
from ciervo.io import load_data
from ciervo.models import label_data, train_test_split
from tqdm import tqdm

Requirement already up-to-date: ciervo in /opt/conda/lib/python3.6/site-packages (2024.8.30)
importing Jupyter notebook from functions.ipynb
Requirement already up-to-date: ciervo in /opt/conda/lib/python3.6/site-packages (2024.8.30)
fatal: destination path 'balu3' already exists and is not an empty directory.
Processing ./balu3
  Created wheel for balu3: filename=balu3-1.0-cp36-none-any.whl size=43719 sha256=d24741c6a5a6a4dde0cf16dcf6cf84db87884d9eb30cb0a003767c755c2d3700
  Stored in directory: /tmp/pip-ephem-wheel-cache-ahq3rgx0/wheels/a0/cb/32/94ff1ea5874a8b4908404cf9d18fac5aec5e0e168cc5a148c1
Successfully built balu3
  Found existing installation: balu3 1.0
    Uninstalling balu3-1.0:
      Successfully uninstalled balu3-1.0


In [2]:
data_files = load_data('data/marcha_larga')

Total time: 2.29 hours


In [12]:
def evaluate_model(data_files, window_size, overlap, num_fases):
    # Cargar y preparar los datos
    labeled_data = label_data(data_files, num_fases=num_fases)
    
    # Dividir los datos en entrenamiento y prueba
    print(len(labeled_data[0].columns))
    train_window, train_labels, test_window, test_labels = train_test_split(
        labeled_data,
        columna=["EMG_Isquio", "EMG_Cuadriceps", "EMG_AductorLargo"],
        window_size=window_size,
        test_size=0.2,
        overlap=overlap,
        random_state=42
    )
    
    # Procesar los datos
    train_data = label_data_and_features(train_window, divide=3)
    test_data = label_data_and_features(test_window, divide=3)
    
    # Seleccionar características
    train_data, test_data = sfs_selection(train_data, test_data, train_labels, n_indices=5)
    
    # Entrenar y evaluar Simple NN
    input_size = train_data.shape[1]
    hidden_size = 100
    output_size = num_fases
    learning_rate = 0.001
    num_epochs = 100
    
    model = SimpleNN(input_size, hidden_size, output_size)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    train_loader, test_tensor, test_labels_tensor = prepare_data_SimpleNN(train_data, train_labels, test_data, test_labels)
    train_model_SimpleNN(model, criterion, optimizer, train_loader, num_epochs)
    accuracy_nn = evaluate_model_SimpleNN(model, test_tensor, test_labels_tensor)
    
    return accuracy_nn

In [13]:
param_grid = {
    'window_size': [ 500],
    'overlap': [0, 50],
    'num_fases': [4, 8, 16]
}

In [14]:
best_accuracy = 0
best_params = {}

for params in ParameterGrid(param_grid):
    print(f"Testing with parameters: {params}")
    accuracy = evaluate_model(data_files,
        window_size=params['window_size'],
        overlap=params['overlap'],
        num_fases=params['num_fases']
    )
    
    print(f"Accuracy: {accuracy}")
    
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_params = params

print("Best parameters found:")
print(best_params)
print("Best accuracy achieved:")
print(best_accuracy)


Testing with parameters: {'num_fases': 4, 'overlap': 0, 'window_size': 500}
12


ValueError: cannot copy sequence with size 500 to array axis with dimension 3

In [ ]:
data_files = load_data('data/marcha_larga')

# Parámetros para la búsqueda en cuadrícula
fases_to_test = [4, 8, 16]
overlaps = [0, 50]
window_sizes = [0.25, 0.5, 1, 2]  # en segundos
SAMPLE_RATE=250
# Convertir window_sizes de segundos a número de muestras
window_sizes_samples = [int(size * SAMPLE_RATE) for size in window_sizes]

# Crear listas para almacenar los resultados
results_simple_nn = []
results_cnn = []
results_rnn = []

# Generar todas las combinaciones de parámetros
param_grid = {
    'fases': fases_to_test,
    'overlap': overlaps,
    'window_size': window_sizes_samples
}

grid = ParameterGrid(param_grid)

for params in grid:
    num_fases = params['fases']
    overlap = params['overlap']
    window_size = params['window_size']
    
    print(f"TESTING WITH Fases: {num_fases}, Overlap: {overlap}, Window Size: {window_size} samples...")
    
    labeled_data = label_data(data_files, num_fases=num_fases)
    
    train_window, train_labels, test_window, test_labels = train_test_split(
        labeled_data, 
        columna=["EMG_Isquio","EMG_Cuadriceps","EMG_AductorLargo"],
        window_size=window_size,
        test_size=0.2,
        overlap=overlap,
        random_state=42
    )
    
    print(f"Train data: Extracted {train_window.shape[0]} samples with {train_window.shape[1]} features each.")
    print(f"Test data: Extracted {test_window.shape[0]} samples with {test_window.shape[1]} features each.")
    
    train_data = label_data_and_features(train_window, divide=3)
    test_data = label_data_and_features(test_window, divide=3)
    
    train_data, test_data = sfs_selection(train_data, test_data, train_labels, n_indices=5)
    print(f"Train data: Extracted {train_data.shape[0]} samples with {train_data.shape[1]} features each.")
    print(f"Test data: Extracted {test_data.shape[0]} samples with {test_data.shape[1]} features each.")
    print("-----------------------------------------------------------------------")
    
    # Simple NN
    print("Simple NN")
    input_size = train_data.shape[1]
    hidden_size = 100
    output_size = num_fases
    learning_rate = 0.001
    num_epochs = 100
    
    model = SimpleNN(input_size, hidden_size, output_size)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    train_loader, test_tensor, test_labels_tensor = prepare_data_SimpleNN(train_data, train_labels, test_data, test_labels)
    train_model_SimpleNN(model, criterion, optimizer, train_loader, num_epochs)
    accuracy_nn = evaluate_model_SimpleNN(model, test_tensor, test_labels_tensor)
    results_simple_nn.append((params, accuracy_nn))
    
    # CNN
    print("-----------------------------------------------------------------------")
    print("CNN")
    train_loader, test_loader, test_labels_tensor, train_data_tensor = prepare_data_CNN(train_data, train_labels, test_data, test_labels)
    input_length = train_data_tensor.shape[2]
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    cnn_model = GaitCNN(input_length, num_fases=num_fases).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(cnn_model.parameters(), lr=0.001)
    
    train_loss, test_loss = train_model_CNN(cnn_model, device, criterion, optimizer, train_loader, test_loader, num_epochs=100)
    accuracy_cnn = evaluate_model_CNN(cnn_model, test_loader, test_labels_tensor)
    results_cnn.append((params, accuracy_cnn))
    
    # RNN
    print("-----------------------------------------------------------------------")
    print("RNN")
    train_data_rnn, test_data_rnn = preprocess_data_RNN(train_data, test_data)
    input_size = train_data_rnn.shape[2]
    hidden_size = 64
    num_layers = 2
    batch_size = 32
    num_epochs = 150
    
    train_loader, test_loader = create_dataloaders_RNN(train_data_rnn, train_labels, test_data_rnn, test_labels, batch_size)
    rnn_model = GaitRNN(input_size, hidden_size, num_layers, num_fases).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(rnn_model.parameters(), lr=0.001)
    
    train_loss, test_loss = train_model_RNN(rnn_model, train_loader, test_loader, criterion, optimizer, num_epochs, device)
    accuracy_rnn = evaluate_model_RNN(rnn_model, test_loader, test_labels, device)
    results_rnn.append((params, accuracy_rnn))
    print("-----------------------------------------------------------------------")

# Mostrar los resultados
print("Results for Simple NN:")
for params, acc in results_simple_nn:
    print(f"Params: {params}, Accuracy: {acc}")

print("Results for CNN:")
for params, acc in results_cnn:
    print(f"Params: {params}, Accuracy: {acc}")

print("Results for RNN:")
for params, acc in results_rnn:
    print(f"Params: {params}, Accuracy: {acc}")
